# Created on: 14-03-2021 10:28 Hrs

# @author: Bernard Birendra Das

# Importing packages

In [ ]:
# imports
import tweepy, os, re
from textblob import TextBlob
from wordcloud import WordCloud
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Uploading Twitter API Credentials

In [ ]:
api_cred = files.upload()

# Loading the API Data

In [ ]:
log = pd.read_csv('api-cred.csv')
consumer_key        =  log['key'][0]
consumer_secret     =  log['key'][1]
access_token        =  log['key'][2]
access_token_secret =  log['key'][3]

# Initializing Classes and Creating Objects

In [ ]:
# driver code
 
# Creating an authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
 
# Set the access_token and access_token_secret
auth.set_access_token(access_token, access_token_secret)
 
# Creating the api object while passing in  the auth information
api =  tweepy.API(auth, wait_on_rate_limit = True)

# Printing the first 100 tweets of the user

In [ ]:
# Extract 100 tweets from twitter.com
posts = api.user_timeline(screen_name = str(input("Enter the name of the account:\t@")), count= 3000, tweet_mode = "extended")
# for tweet in tweepy.Cursor(api.user_timeline,id='USATODAY').items():
#    print(tweet)
 
# Print the last 5 tweets from the account
print("Show the 5 recent tweets:\n")
i = 1
for tweet in posts[0:3000]:
  print("Tweet ["+str(i)+"] : "+tweet.full_text + "\n")
  i+=1

# Creating our Dataframe

In [ ]:
# Create a dataframe with a column called Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])
 
# Show the first 5 rows of data
df.head()

# Cleaning the Dataframe

In [ ]:
# Clean the data
 
# Function to clean the tweets
def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text)   # Simply removes @mentions
  text = re.sub(r'#', '', text)               # Simply removes '#' symbols
  text = re.sub(r'\n', '', text)              # Simply removes newline characters
  text = re.sub(r'RT[\s]+', '', text)         # Simply removes Retweets
  text = re.sub(r'https?:\/\/\S+', '', text)  # Simply removes hyperlinks
  text = re.sub(r'অ-', '', text)  # Simply removes hyperlinks
 
  return text
 
# Cleaning the Tweets
df['Tweets'] = df['Tweets'].apply(cleanTxt)
  
# Show Cleaned Tweets
df

# Determining the polarity and subjectivity of the tweets

In [ ]:
# Create a function to get the subjectivity
def getSub(text):
  return TextBlob(text).sentiment.subjectivity
 
# Create a function to get the polarity
def getPol(text):
  return TextBlob(text).sentiment.polarity
 
# Create two new columns for subjectivity and polarity
df['Subjectivity'] = df['Tweets'].apply(getSub)
df['Polarity'] = df['Tweets'].apply(getPol)
 
# Display the new columns
df

# Plotting a Word Cloud on the most appeared words

In [ ]:
# plot word cloud
allWords = ' '.join([twts for twts in df['Tweets']])
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 120).generate(allWords)
 
plt.imshow(wordCloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

# Categorizing the tweets as positive, negative and neutral

In [ ]:
# Create a function to compute the negative, neutral and positive analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'
 
df['Analysis'] = df['Polarity'].apply(getAnalysis)
 
# Show the dataframe
df

# Printing all the Positive Tweets

In [ ]:
# Print all of the positive tweets
j = 1
sortedDF = df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Positive'):
    print("["+str(j)+"] "+sortedDF['Tweets'][i])
    print()
    j+=1

# Print all of the Negative Tweets

In [ ]:
# Print all of the negative tweets
j = 1
sortedDF = df.sort_values(by=['Polarity'], ascending = 'False')
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == 'Negative'):
    print("["+str(j)+"] "+sortedDF['Tweets'][i])
    print()
    j+=1

# Plotting the polarity and subjectivity

In [ ]:
# Plot the polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range (0, df.shape[0]):
  plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color = 'Blue')
 
plt.title('Sentiment-Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')

# Determinig the percentage of positive, negative and neutral tweets

In [ ]:
# get the percentage of positive tweets and negative tweets
ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['Tweets']
 
# get the percentage of negative tweets
ntweets = df[df.Analysis == 'Negative']
ntweets = ntweets['Tweets']
 
ptn = round((ptweets.shape[0] / df.shape[0])*100, 0)
ntn = round((ntweets.shape[0] / df.shape[0])*100, 0)
neu = round((100 - (ptn + ntn)), 0)
 
print("POSITIVE TWEETS:\t"+str(ptn)+"% [+]")
print("NEGATTIVE TWEETS:\t"+str(ntn)+"% [-]")
print("NEUTRAL TWEETS:\t\t"+str(neu)+"% [~]")

# Plotting the sentiments on a BAR GRAPH

In [ ]:
# Show the value counts
df['Analysis'].value_counts()
 
# plot and visualize the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show

# Plotting the sentiments on a PIE CHART

In [ ]:
# Show the value counts
df['Analysis'].value_counts()
 
# plot and visualize the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='pie')
plt.show

# Plotting the sentiments on a LINE GRAPH

In [ ]:
# Show the value counts
df['Analysis'].value_counts()
 
# plot and visualize the counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='line')
plt.show